# 1. Passo a passo

1. Baixar base de dados
2. Criar as amostras com reposição e colocar numa lista
3. Para cada amostra, dividir em treino e teste
4. Para cada amostra, treinar o modelo
5. Para cada amostra, criar uma previsões de teste.
6. Juntar todas as previsões em um dataframe.
7. Fazer a agregação a partir das frequências

# 2. Descrição

O Bagging é um processo de junção de modelos para melhorar a eficácia do resultado. Consiste em se criar amostras variadas com repetição em cima do conjunto de dados inicial. Cada amostra será utilizado para treinar um modelo e cada modelo será utilizado para se tirar previsões. À partir dessas previsões, utilizamos uma ténica de agregação para entender qual o melhor resultado. No caso de variáveis discretas, o melhor resultado é o que tiver maior frequência entre os diferentes modelos. Para variáveis contínuas, o melhor resultado é a média de todos os resultados.

In [49]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [42]:
df = sns.load_dataset("penguins")
df.dropna(inplace=True)
df['species'] = df['species'].map({'Adelie': 0, 'Chinstrap':1, 'Gentoo': 2 })
df['sex'] = df['sex'].map({'Male':0, 'Female':1})
df



,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,0,Torgersen,39.1,18.7,181.0,3750.0,0
1,0,Torgersen,39.5,17.4,186.0,3800.0,1
2,0,Torgersen,40.3,18.0,195.0,3250.0,1
4,0,Torgersen,36.7,19.3,193.0,3450.0,1
5,0,Torgersen,39.3,20.6,190.0,3650.0,0
...,...,...,...,...,...,...,...
338,2,Biscoe,47.2,13.7,214.0,4925.0,1
340,2,Biscoe,46.8,14.3,215.0,4850.0,1
341,2,Biscoe,50.4,15.7,222.0,5750.0,0
342,2,Biscoe,45.2,14.8,212.0,5200.0,1


In [65]:
amostras = []
for i in range(20):
    amostras.append(df.sample(len(df), replace=True, random_state=i))
    
amostras

[     species     island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
 178        1      Dream            50.5           18.4              200.0   
 53         0     Biscoe            42.0           19.5              200.0   
 123        0  Torgersen            41.4           18.5              202.0   
 198        1      Dream            50.1           17.9              190.0   
 332        2     Biscoe            43.5           15.2              213.0   
 ..       ...        ...             ...            ...                ...   
 233        2     Biscoe            48.4           14.6              213.0   
 281        2     Biscoe            46.2           14.9              221.0   
 298        2     Biscoe            45.2           13.8              215.0   
 57         0     Biscoe            40.6           18.8              193.0   
 164        1      Dream            47.0           17.3              185.0   
 
      body_mass_g  sex  
 178       3400.0    1  
 53        4

In [66]:
splits = []
for amostra in amostras:
    #print(amostra['island'])
    X = amostra.drop(['species', 'island'], axis=1)
    y = amostra['species']
    splits.append(train_test_split(X, y, test_size=0.3))
    
splits[0]

[     bill_length_mm  bill_depth_mm  flipper_length_mm  body_mass_g  sex
 186            49.7           18.6              195.0       3600.0    0
 14             34.6           21.1              198.0       4400.0    0
 342            45.2           14.8              212.0       5200.0    1
 213            46.8           16.5              189.0       3650.0    1
 153            50.0           19.5              196.0       3900.0    0
 ..              ...            ...                ...          ...  ...
 316            49.4           15.8              216.0       4925.0    0
 49             42.3           21.2              191.0       4150.0    0
 133            37.5           18.5              199.0       4475.0    0
 123            41.4           18.5              202.0       3875.0    0
 265            43.6           13.9              217.0       4900.0    1
 
 [233 rows x 5 columns],
      bill_length_mm  bill_depth_mm  flipper_length_mm  body_mass_g  sex
 315            50.8    

In [67]:
models=[]

for split in splits:
    model = DecisionTreeClassifier(max_depth=5, min_samples_leaf=20)
    model.fit(split[0], split[2])
    models.append(model)
    
models

[DecisionTreeClassifier(max_depth=5, min_samples_leaf=20),
 DecisionTreeClassifier(max_depth=5, min_samples_leaf=20),
 DecisionTreeClassifier(max_depth=5, min_samples_leaf=20),
 DecisionTreeClassifier(max_depth=5, min_samples_leaf=20),
 DecisionTreeClassifier(max_depth=5, min_samples_leaf=20),
 DecisionTreeClassifier(max_depth=5, min_samples_leaf=20),
 DecisionTreeClassifier(max_depth=5, min_samples_leaf=20),
 DecisionTreeClassifier(max_depth=5, min_samples_leaf=20),
 DecisionTreeClassifier(max_depth=5, min_samples_leaf=20),
 DecisionTreeClassifier(max_depth=5, min_samples_leaf=20),
 DecisionTreeClassifier(max_depth=5, min_samples_leaf=20),
 DecisionTreeClassifier(max_depth=5, min_samples_leaf=20),
 DecisionTreeClassifier(max_depth=5, min_samples_leaf=20),
 DecisionTreeClassifier(max_depth=5, min_samples_leaf=20),
 DecisionTreeClassifier(max_depth=5, min_samples_leaf=20),
 DecisionTreeClassifier(max_depth=5, min_samples_leaf=20),
 DecisionTreeClassifier(max_depth=5, min_samples_leaf=20

In [117]:
test_X= splits[0][1].sample(1, random_state=241)
test_X

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
305,50.8,17.3,228.0,5600.0,0


In [118]:
predictions = []
for model in models:
    y_pred = model.predict(test_X)
    predictions.append(y_pred[0])
predictions

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2]

In [119]:
final_prediction = pd.DataFrame(predictions).value_counts().idxmax()[0]
final_prediction

2